# Cholesky decomposition rank-one update

## implementation

In [1]:
using LinearAlgebra

In [3]:
"""
    CholeskyDecompRank1Update(L, x)

Calculate the Cholesky decomposition of `LL' + xx'` with `O(n)` computational effort, where `L` is a `n x n` lower-triangular invertible matrix, and `x` is a `n`-dimentional vector.
The inputs `L` and `x` are modified in place.
"""
function CholeskyDecompRank1Update(L::LowerTriangular{T1, T_mat}, x::T_vec) where {T1<:Union{AbstractFloat, Complex}, T2<:Union{AbstractFloat, Complex}, T_mat<:AbstractMatrix{T1}, T_vec<:AbstractVector{T2}}
    N = length(x)

    @assert N > 0
    @assert size(L) == (N,N)

    T_out = promote_type(T1, T2)

    for i = 1:N-1
        r = sqrt(abs2(L[i,i]) + abs2(x[i]))

        vec_f = zeros(T_out, N)
        vec_f[i] = r
        vec_f[i+1:end] = (L[i,i]*L[i+1:end,i] + conj(x[i])*x[i+1:end]) / r

        x[i+1:end] = (L[i,i]*x[i+1:end] - x[i]*L[i+1:end,i]) / r

        L[i:end,i] = vec_f[i:end]
    end

    L[N,N] = sqrt(abs2(L[N,N]) + abs2(x[N]))
    return L
end

CholeskyDecompRank1Update

## test

In [4]:
let
    # Create a random Hermitian positive-define matrix.
    N = 100
    A = rand(N,N) + im*rand(N,N)
    A = A'*A + Matrix(I, N,N)

    # Calculate the Cholesky decomposition of `A`
    Chol_A = cholesky(A)
    L = Chol_A.L

    # Create a random vector.
    x = rand(N) + im*rand(N)

    # Create an updated matrix
    A2 = A + x*x'

    # Update the existing Cholesky decomposition.
    L2 = deepcopy(L)
    L2 = CholeskyDecompRank1Update(L2, x)

    # Verify the result.
    A2_pred = L2*L2'
    M_diff = A2_pred - A2
    println("maximum abs error: $(maximum(abs.(M_diff)))")
end

maximum abs error: 9.237055564881302e-14


## Playground

In [28]:
let
    # Create a random Hermitian positive-define matrix.
    N = 5
    A = rand(N,N) + im*rand(N,N)
    A = A'*A + Matrix(I, N,N)
    println("A:")
    display(A)
    #println("eig vals of A:")
    #display(eigvals(A))

    # Calculate the Cholesky decomposition of `A`
    Chol_A = cholesky(A)
    L = Chol_A.L

    # Create a random vector.
    x = rand(N) + im*rand(N)
    println("x:")
    display(x)

    # Create an updated matrix
    A2 = A + x*x'
    println("A2 (A + xx'):")
    display(A2)

    # Update the existing Cholesky decomposition.
    L2 = deepcopy(L)
    L2 = CholeskyDecompRank1Update(L2, x)
    println("L2 (CholeskyDecompRank1Update(L, x)):")
    display(L2)

    # Verify the result.
    A2_pred = L2*L2'
    #println("A2_pred:")
    #display(A2_pred)
    M_diff = A2_pred - A2
    #println("M_diff:")
    #display(M_diff)
    println("maximum abs error: $(maximum(abs.(M_diff)))")
end

A:


5×5 Matrix{ComplexF64}:
 4.61013+0.0im       2.63253+0.716959im  …  2.08976-0.602582im
 2.63253-0.716959im  3.65244+0.0im          1.18828-0.9492im
 1.85733-0.364287im  1.61213-0.23294im      1.04994+0.420297im
 2.09211-0.239236im  1.18583+0.385239im     1.73521-0.629437im
 2.08976+0.602582im  1.18828+0.9492im       3.00451+0.0im

x:


5-element Vector{ComplexF64}:
 0.47807696748186523 + 0.22086892558135485im
   0.576760882312709 + 0.7108521932331551im
 0.23674278356103506 + 0.17348355647613278im
  0.5302429832182669 + 0.7640038384282584im
  0.8255974470137991 + 0.3845861047314768im

A2 (A + xx'):


5×5 Matrix{ComplexF64}:
 4.88747+0.0im         3.06528+0.504505im  …  2.56941-0.604095im
 3.06528-0.504505im     4.4904+0.0im          1.93784-0.584136im
 2.00883-0.333638im    1.87199-0.30117im      1.31211+0.472476im
 2.51435+0.00890195im  2.03474+0.448962im     2.46681-0.202602im
 2.56941+0.604095im    1.93784+0.584136im     3.83403+0.0im

L2 (CholeskyDecompRank1Update(L, x)):


5×5 LowerTriangular{ComplexF64, Matrix{ComplexF64}}:
 2.21076+0.0im         …           ⋅                    ⋅    
 1.38652-0.228204im                ⋅                    ⋅    
 0.90866-0.150915im                ⋅                    ⋅    
 1.13732+0.00402664im      1.44478+0.0im                ⋅    
 1.16223+0.273252im       0.553657-0.0904118im  1.31994+0.0im

maximum abs error: 8.881784197001252e-16
